In [140]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
import address_wgs1984 as ad

In [172]:
def house_information(url):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.80 Safari/537.36"}
    b = requests.get(url,headers=headers)
    data = []
    if b.status_code == 200:
        html = b.text
        soup = BeautifulSoup(html, 'lxml')
    
    community = soup.find_all(name='a',attrs={"class": "blue-14"})[0].text  # 社区名称
    
    try:
        price1 = int(soup.find_all(name='strong')[0].text)
        price2 = int(soup.find_all(name='strong')[1].text)
    except:
        price1 = "无数据"
        price2 = "无数据"
    
    type1 = soup.find_all(name='li',attrs={"class": "one"})[0].find_all(name="strong")[0].text   #几房
    height = soup.find_all(name='li',attrs={"class": "one"})[0].find_all(name="p")[0].text.split("/")   #层数
    area = soup.find_all(name='li',attrs={"class": "two"})[0].find_all(name="strong")[0].text.replace("㎡","")  #面积
    type2 = soup.find_all(name='li',attrs={"class": "two"})[0].find_all(name="p")[0].text   #类型2
    direction = soup.find_all(name='li',attrs={"class": "three"})[1].find_all(name="strong")[0].text #方向
    type3 = soup.find_all(name='li',attrs={"class": "three"})[1].find_all(name="p")[0].text #类型3
    
    address = soup.find_all(name='div',attrs={"class": "esf-info-box b-b-dc fix otherinfo f14"})[0].find_all("p")[1].text.replace("所在地址：","").replace(" 查看地图","")
    #location = ad.geo_address(address,"深圳")
    
    return {"community":community,
        "all_price(万元)":price1,
        "gov_price(/㎡)":price2,
        "area(㎡)":area,
        "number":type1,
        "fitment":type3,
        "type":type2,
        "floor":height[0],
        "all_floor":height[1],
        "direction":direction,
        "address":address}

In [176]:
page= 1
data = []
for i in tqdm(range(0,94)):
    page += 1 
    page_url = "http://zf.szhome.com/Search.html?sor=1&aom=1&kwd=&xzq=0&pq=0&price=0&prif=0&prit=0&barea=0&baf=0&bat=0&hx=0&ord=0&dtyx=0&dtst=0&scat=0&sx=0&schid=0&page=" + str(page)
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.80 Safari/537.36"}
    b = requests.get(page_url,headers=headers)
    if b.status_code == 200:
        html = b.text
        soup = BeautifulSoup(html, 'lxml')
        url_database = soup.find_all(name='div',attrs={"class": "lpinfo"})
        for i in url_database:
            url = "http://zf.szhome.com" + i.find_all("a")[0].attrs["href"]
            data.append(house_information(url))

100%|██████████| 94/94 [06:39<00:00,  4.25s/it]


In [177]:
df = pd.DataFrame(columns=["community","all_price(万元)","gov_price(/㎡)","area(㎡)","number","fitment","type","floor","all_floor","direction",
                           "address"])  #构建一个空表

In [178]:
df = df.append(data,ignore_index=True)
df.to_excel("price_data.xlsx") 